In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# jax
import jax.numpy as jnp
from jax import random
import time

# stats
from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm

# interpolation
from scipy.interpolate import CubicSpline

# pre-processing
from sklearn.preprocessing import StandardScaler

# collocation
import sys
import os
p_ = os.path.abspath(os.path.join('..', '00_utils'))
if p_ not in sys.path:
    sys.path.append(p_)
    
p_ = os.path.abspath(os.path.join('..', '00_models'))
if p_ not in sys.path:
    sys.path.append(p_)

from collocation import compute_weights, lagrange_derivative
from non_parametric_collocation import collocate_data

import preprocess # helper preprocessing class
import ode_solver_pyomo_base

# pyomo
import importlib
import nn_pyomo_base
import nn_pyomo_base_dae

import statsmodels.api as sm
from scipy.ndimage import gaussian_filter1d

In [12]:
# reload the necessary libraries
importlib.reload(nn_pyomo_base)
NeuralODEPyomo = nn_pyomo_base.NeuralODEPyomo

importlib.reload(preprocess)
DataPreprocessor = preprocess.DataPreprocessor

importlib.reload(ode_solver_pyomo_base)
DirectODESolver = ode_solver_pyomo_base.DirectODESolver

# set up data parameters
file_path = '../00_data/df_train.csv'
encoding = {'settlement_date': 't', 'temperature': 'var1', 'hour': 'var2', 'nd': 'y'}

# fix model parameters 
layer_sizes = [7, 30, 1]

# fix optimal parameters
tol = 1e-8
params_nn = {"tol":tol, 
        "dual_inf_tol": tol, "compl_inf_tol": tol, "constr_viol_tol": tol, 
        "halt_on_ampl_error" : 'yes', "print_level": 1, "max_iter": 500}

tol = 1e-7
params_ode = {"tol":1e-7, 
        "dual_inf_tol": tol, "compl_inf_tol": tol, "constr_viol_tol": tol, 
        "acceptable_iter": 0,  
        "halt_on_ampl_error" : 'yes',
        "print_level": 1, 
        "max_iter": 100 }

In [13]:
from datetime import datetime, timedelta

# prepare data ranges
START_DATE = '2015-01-10'
start_date = datetime.strptime(START_DATE, '%Y-%m-%d')
date_sequences = [start_date + timedelta(days=i*5) for i in range(10)]
date_sequences_str = [date.strftime('%Y-%m-%d') for date in date_sequences]
print(date_sequences_str)

times_elapsed = []

['2015-01-10', '2015-01-15', '2015-01-20', '2015-01-25', '2015-01-30', '2015-02-04', '2015-02-09', '2015-02-14', '2015-02-19', '2015-02-24']


In [24]:
experiment_results = {}
START_DATE = '2015-01-10'

for START_DATE in date_sequences_str:
    data_loader = DataPreprocessor(file_path, start_date = START_DATE, number_of_points = 400, tau = 48, m = 1, feature_encoding = encoding, split=200)
    data_subsample = data_loader.load_data()
    df_train, df_test = data_loader.preprocess_data(data_subsample)

    ys = np.atleast_2d(df_train['y']).T
    ts = np.array(df_train['t'])
    Xs = np.atleast_2d(df_train.drop(columns=['y', 't']))

    w = compute_weights(df_train['t'])
    Ds = np.array(lagrange_derivative(df_train['t'], w))

    ode_model = NeuralODEPyomo(y_observed = ys, 
                            t = ts, # t
                            first_derivative_matrix = Ds, # derivative matrix
                            extra_input = Xs, # extra inputs
                            y_init = ys,
                            layer_sizes = layer_sizes, act_func = "tanh", 
                            penalty_lambda_reg = 0.01, 
                            time_invariant = True,
                            w_init_method = 'xavier', 
                            params = params_nn
                            )

    ode_model.build_model()
    result = ode_model.solve_model()
    u_model = ode_model.extract_solution().T
    
    experiment_results[START_DATE] = {}
    experiment_results[START_DATE]['times_elapsed'] = result['solver_time']
    
    # ---------------------------------------------- ODEINT PREDICTION ----------------------------------------------
    y0 = ys[0]
    y_pred = ode_model.neural_ode(y0, ts, (Xs, ts))

    plt.figure(figsize=(20, 10))
    plt.plot(ts, ys, label='True Data', alpha = 1)
    plt.plot(ts, u_model, label='True Data', alpha = 1)
    plt.scatter(ts, y_pred, label='Model Prediction (Train) - Odeint', alpha = 0.7)
    plt.title(f"Sequential ODE solver Result; Time Elapsed: {result['solver_time']}")
    plt.legend(loc ="lower right")
    plt.grid(True)
    plt.savefig(f'plots/ode_solver_train_{START_DATE}.png', format='png')  
    plt.close() 
    
    # -------------------------------------------- COLLOCATION PREDICTION ---------------------------------------------- 
    # extract weights & biases
    trained_weights_biases = ode_model.extract_weights()
    W1, b1 = trained_weights_biases['W1'], trained_weights_biases['b1']
    W2, b2 = trained_weights_biases['W2'], trained_weights_biases['b2']
    trained_weights, trained_biases = [W1, W2], [b1, b2]     
    
    # set up and solve the model
    initial_state = u_model[0]
    direct_solver = DirectODESolver(np.array(ts), layer_sizes, trained_weights, trained_biases, initial_state, 
                                    D = Ds, time_invariant=True, extra_input=np.array(Xs), params = params)
    direct_solver.build_model()
    solver_info = direct_solver.solve_model()
    y_solution = direct_solver.extract_solution()     
    
    plt.figure(figsize=(20, 10))
    plt.plot(ts, ys, label='True Data', alpha = 1)
    plt.plot(ts, u_model, label='True Data', alpha = 1)
    plt.plot(ts, y_solution, label='Model Prediction (Train) - Collocation-based ODE', alpha = 0.7)
    plt.title(f"Collocation-based ODE solver Result; Time Elapsed: {result['solver_time']}")
    plt.legend(loc ="lower right")
    plt.grid(True)
    plt.savefig(f'plots/collocation_solver_train_{START_DATE}.png', format='png')  
    plt.close() 
    
    ys_test = np.atleast_2d(df_test['y']).T
    ts_test = np.array(df_test['t'])
    Xs_test = np.atleast_2d(df_test.drop(columns=['y', 't']))
    w_test = compute_weights(df_test['t'])
    Ds_test = np.array(lagrange_derivative(df_test['t'].values, w_test))
    
    initial_state = ys_test[0][0]
    direct_solver = DirectODESolver(ts_test, layer_sizes, trained_weights, trained_biases, initial_state, 
                                    D = Ds_test,
                                    time_invariant=True, extra_input=np.array(Xs_test), params = params)
    direct_solver.build_model()
    
    solver_info = direct_solver.solve_model()
    y_solution = direct_solver.extract_solution() 
    
    plt.figure(figsize=(20, 10))
    plt.plot(ts_test, ys_test)
    plt.plot(ts_test, y_solution, label='Model Prediction (Test) -  Collocation-based ODE', alpha = 0.7)
    plt.title(f"Collocation-based ODE solver Result (Test); Time Elapsed: {result['solver_time']}")
    plt.legend(loc ="lower right")
    plt.grid(True)
    plt.savefig(f'plots/collocation_solver_test_{START_DATE}.png', format='png')  
    plt.close() 

y
Ipopt 3.14.16: tol=1e-08
dual_inf_tol=1e-08
compl_inf_tol=1e-08
constr_viol_tol=1e-08
halt_on_ampl_error=yes
print_level=5
max_iter=500


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.1.

Number of nonzeros in equality constraint Jacobian...:    93729
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:    55281

Total number of variables............................:      471
                     variables with only lower bounds:        0
                variables with lower and upper bounds:      471
                 

In [25]:
experiment_results

{'2015-01-10': {'times_elapsed': 7.035712003707886},
 '2015-01-15': {'times_elapsed': 6.586349964141846},
 '2015-01-20': {'times_elapsed': 4.4962239265441895},
 '2015-01-25': {'times_elapsed': 2.6127960681915283},
 '2015-01-30': {'times_elapsed': 5.528671979904175},
 '2015-02-04': {'times_elapsed': 3.5849459171295166},
 '2015-02-09': {'times_elapsed': 3.5727250576019287},
 '2015-02-14': {'times_elapsed': 3.446420192718506},
 '2015-02-19': {'times_elapsed': 5.0869460105896},
 '2015-02-24': {'times_elapsed': 5.859051942825317}}